## Get file

In [100]:
file_path = "C:/Users/nirutcha/Documents/GitHub/CleansingData/Data/20240829CAR_MODEL_LIST.xlsx"

## Craete SparkSession

In [101]:
import pandas as pd
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("Excel to Spark").getOrCreate()

In [102]:
pdf = pd.read_excel(file_path, sheet_name='Sheet1')

In [103]:
df = spark.createDataFrame(pdf)

In [104]:
df.show(truncate=False)

+----------+-----------------------+----------------------+----------------------+
|Unnamed: 0|CAR_BRAND_CAR_MODEL_CAR|CAR_BRAND_CAR_MODEL_SG|CAR_BRAND_CAR_MODEL_MD|
+----------+-----------------------+----------------------+----------------------+
|0         |MG_MG 3                |ISUZU_D-MAX           |VOLVO_144             |
|1         |TOYOTA_INOVA           |TOYOTA_VIGO           |VOLVO_164             |
|2         |MAZDA_3                |TOYOTA_VIOS           |VOLVO_240             |
|3         |AUDI_TT                |HONDA_CITY            |VOLVO_244             |
|4         |TOYOTA_MIGHTY-X        |HONDA_CIVIC           |VOLVO_260             |
|5         |ISUZU_DMAX             |TOYOTA_ALTIS          |VOLVO_264             |
|6         |HONDA_CR-V             |TOYOTA_REVO           |VOLVO_340             |
|7         |NISSAN_ ELGRAND        |TOYOTA_YARIS          |VOLVO_446             |
|8         |MAZDA_MAZDA2           |HONDA_JAZZ            |VOLVO_460             |
|9  

In [105]:
df.printSchema()

root
 |-- Unnamed: 0: long (nullable = true)
 |-- CAR_BRAND_CAR_MODEL_CAR: string (nullable = true)
 |-- CAR_BRAND_CAR_MODEL_SG: string (nullable = true)
 |-- CAR_BRAND_CAR_MODEL_MD: string (nullable = true)



In [106]:
print(df.count(),len(df.columns))

3067 4


In [107]:
df.summary('count').show()

+-------+----------+-----------------------+----------------------+----------------------+
|summary|Unnamed: 0|CAR_BRAND_CAR_MODEL_CAR|CAR_BRAND_CAR_MODEL_SG|CAR_BRAND_CAR_MODEL_MD|
+-------+----------+-----------------------+----------------------+----------------------+
|  count|      3067|                   3067|                  3067|                  3067|
+-------+----------+-----------------------+----------------------+----------------------+



In [108]:
df.select('CAR_BRAND_CAR_MODEL_MD').distinct().count()

1055

## Cleansing

#### Craete new column under condition replace as MD if fuzz.ration >= 90

In [167]:
from pyspark.sql.functions import udf, col, lit
from pyspark.sql.types import StringType
from fuzzywuzzy import fuzz

In [222]:
def get_best_match(sg_value, md_list):
    if sg_value is None:
        return None
    
    best_match = None
    highest_ratio = 0
    
    for md_value in md_list:
        ratio = fuzz.ratio(sg_value, md_value)
        if ratio >= 90 and ratio > highest_ratio:
            highest_ratio = ratio
            best_match = md_value
    
    if best_match is not None:
        return best_match
    else:
        return sg_value

@udf(StringType())
def fuzzy_match_udf(sg_value, md_list):
    return get_best_match(sg_value, md_list)

In [223]:
md_list = df.select('CAR_BRAND_CAR_MODEL_MD').rdd.flatMap(lambda x: x).collect()

In [224]:
result_df = df.withColumn('new_col', fuzzy_match_udf(col('CAR_BRAND_CAR_MODEL_SG'), lit(md_list)))


In [225]:
result_df.show()

+----------+-----------------------+----------------------+----------------------+-----------------+
|Unnamed: 0|CAR_BRAND_CAR_MODEL_CAR|CAR_BRAND_CAR_MODEL_SG|CAR_BRAND_CAR_MODEL_MD|          new_col|
+----------+-----------------------+----------------------+----------------------+-----------------+
|         0|                MG_MG 3|           ISUZU_D-MAX|             VOLVO_144|      ISUZU_D-MAX|
|         1|           TOYOTA_INOVA|           TOYOTA_VIGO|             VOLVO_164|      TOYOTA_VIGO|
|         2|                MAZDA_3|           TOYOTA_VIOS|             VOLVO_240|      TOYOTA_VIOS|
|         3|                AUDI_TT|            HONDA_CITY|             VOLVO_244|       HONDA_CITY|
|         4|        TOYOTA_MIGHTY-X|           HONDA_CIVIC|             VOLVO_260|      HONDA_CIVIC|
|         5|             ISUZU_DMAX|          TOYOTA_ALTIS|             VOLVO_264|     TOYOTA_ALTIS|
|         6|             HONDA_CR-V|           TOYOTA_REVO|             VOLVO_340|      TOY

In [226]:
result_df.select('new_col').distinct().count()

2471

In [227]:
result_df.filter(
    (col('CAR_BRAND_CAR_MODEL_SG') == 'SUZU_D-MAX 2.5') |
    (col('CAR_BRAND_CAR_MODEL_SG') == 'ISUZU_D-MAM') |
    (col('CAR_BRAND_CAR_MODEL_SG') == 'ISUZU_-D-MAX')
).show()

+----------+-----------------------+----------------------+----------------------+-----------+
|Unnamed: 0|CAR_BRAND_CAR_MODEL_CAR|CAR_BRAND_CAR_MODEL_SG|CAR_BRAND_CAR_MODEL_MD|    new_col|
+----------+-----------------------+----------------------+----------------------+-----------+
|      1219|        TOYOTA_COROLLLA|          ISUZU_-D-MAX|                   NaN|ISUZU_D-MAX|
|      2026|                    NaN|           ISUZU_D-MAM|                   NaN|ISUZU_D-MAX|
+----------+-----------------------+----------------------+----------------------+-----------+



In [228]:
car_list = df.select('CAR_BRAND_CAR_MODEL_CAR').rdd.flatMap(lambda x: x).collect()

In [229]:
result_df_2 = result_df.withColumn('new_col2', fuzzy_match_udf(col('new_col'), lit(car_list)))


In [230]:
result_df_2.show()

+----------+-----------------------+----------------------+----------------------+-----------------+----------------+
|Unnamed: 0|CAR_BRAND_CAR_MODEL_CAR|CAR_BRAND_CAR_MODEL_SG|CAR_BRAND_CAR_MODEL_MD|          new_col|        new_col2|
+----------+-----------------------+----------------------+----------------------+-----------------+----------------+
|         0|                MG_MG 3|           ISUZU_D-MAX|             VOLVO_144|      ISUZU_D-MAX|     ISUZU_D-MAX|
|         1|           TOYOTA_INOVA|           TOYOTA_VIGO|             VOLVO_164|      TOYOTA_VIGO|    TOYOTA_ VIGO|
|         2|                MAZDA_3|           TOYOTA_VIOS|             VOLVO_240|      TOYOTA_VIOS|    TOYOTA_ VIOS|
|         3|                AUDI_TT|            HONDA_CITY|             VOLVO_244|       HONDA_CITY|     HONDA้_CITY|
|         4|        TOYOTA_MIGHTY-X|           HONDA_CIVIC|             VOLVO_260|      HONDA_CIVIC|    HONDA้_CIVIC|
|         5|             ISUZU_DMAX|          TOYOTA_ALT

#### Craete new column 2 under condition replace as CAR if fuzz.ration >= 90

In [231]:
result_df_2.filter(
    (col('CAR_BRAND_CAR_MODEL_SG') == 'TOYOTA_SOLUNA 1.5GLI') |
     (col('CAR_BRAND_CAR_MODEL_SG') == 'TOYOTA_SOLUNA 1.5') |
     (col('CAR_BRAND_CAR_MODEL_SG') == 'TOYOTA_SOLUNA GLI.') 
     ).show()

+----------+-----------------------+----------------------+----------------------+--------------------+-----------------+
|Unnamed: 0|CAR_BRAND_CAR_MODEL_CAR|CAR_BRAND_CAR_MODEL_SG|CAR_BRAND_CAR_MODEL_MD|             new_col|         new_col2|
+----------+-----------------------+----------------------+----------------------+--------------------+-----------------+
|      1057|               HYUN_H-1|  TOYOTA_SOLUNA 1.5GLI|                   NaN|TOYOTA_SOLUNA 1.5GLI|TOYOTA_SOLUNA GLI|
|      1244|           TOYOTA_FORTU|     TOYOTA_SOLUNA 1.5|                   NaN|   TOYOTA_SOLUNA 1.5|TOYOTA_SOLUNA 1.5|
|      2753|                    NaN|    TOYOTA_SOLUNA GLI.|                   NaN|  TOYOTA_SOLUNA GLI.|TOYOTA_SOLUNA GLI|
+----------+-----------------------+----------------------+----------------------+--------------------+-----------------+



In [241]:
from pyspark.sql.functions import upper, col
result_df_2 = result_df.withColumn('new_col2', upper(col('new_col')))

In [242]:
result_df_2.show()

+----------+-----------------------+----------------------+----------------------+-----------------+-----------------+
|Unnamed: 0|CAR_BRAND_CAR_MODEL_CAR|CAR_BRAND_CAR_MODEL_SG|CAR_BRAND_CAR_MODEL_MD|          new_col|         new_col2|
+----------+-----------------------+----------------------+----------------------+-----------------+-----------------+
|         0|                MG_MG 3|           ISUZU_D-MAX|             VOLVO_144|      ISUZU_D-MAX|      ISUZU_D-MAX|
|         1|           TOYOTA_INOVA|           TOYOTA_VIGO|             VOLVO_164|      TOYOTA_VIGO|      TOYOTA_VIGO|
|         2|                MAZDA_3|           TOYOTA_VIOS|             VOLVO_240|      TOYOTA_VIOS|      TOYOTA_VIOS|
|         3|                AUDI_TT|            HONDA_CITY|             VOLVO_244|       HONDA_CITY|       HONDA_CITY|
|         4|        TOYOTA_MIGHTY-X|           HONDA_CIVIC|             VOLVO_260|      HONDA_CIVIC|      HONDA_CIVIC|
|         5|             ISUZU_DMAX|          TO

### Drop new_col and change column names

In [243]:
result_df_2 = result_df_2.drop('new_col')

In [245]:
result_df_2.columns

['Unnamed: 0',
 'CAR_BRAND_CAR_MODEL_CAR',
 'CAR_BRAND_CAR_MODEL_SG',
 'CAR_BRAND_CAR_MODEL_MD',
 'new_col2']

In [247]:
result_df_2 = result_df_2.toDF('index', 'model_car', 'model_sg', 'model_md', 'model_new')

In [249]:
result_df_2.show()

+-----+-------------------+-----------------+-------------------+-----------------+
|index|          model_car|         model_sg|           model_md|        model_new|
+-----+-------------------+-----------------+-------------------+-----------------+
|    0|            MG_MG 3|      ISUZU_D-MAX|          VOLVO_144|      ISUZU_D-MAX|
|    1|       TOYOTA_INOVA|      TOYOTA_VIGO|          VOLVO_164|      TOYOTA_VIGO|
|    2|            MAZDA_3|      TOYOTA_VIOS|          VOLVO_240|      TOYOTA_VIOS|
|    3|            AUDI_TT|       HONDA_CITY|          VOLVO_244|       HONDA_CITY|
|    4|    TOYOTA_MIGHTY-X|      HONDA_CIVIC|          VOLVO_260|      HONDA_CIVIC|
|    5|         ISUZU_DMAX|     TOYOTA_ALTIS|          VOLVO_264|     TOYOTA_ALTIS|
|    6|         HONDA_CR-V|      TOYOTA_REVO|          VOLVO_340|      TOYOTA_REVO|
|    7|    NISSAN_ ELGRAND|     TOYOTA_YARIS|          VOLVO_446|     TOYOTA_YARIS|
|    8|       MAZDA_MAZDA2|       HONDA_JAZZ|          VOLVO_460|       HOND

## Export

In [250]:
result_df_2_pd = result_df_2.toPandas()

In [251]:
result_df_2_pd.to_excel('output.xlsx')

#### Check ratio

In [253]:
str1 = 'ISUZU_D-MAX ALL NEW'
str2 = 'ISUZU_D-MAX'
fuzz.ratio(str1, str2)

73